This is blog post is to create a python script that acts as a web scrapper. This tool aims to find the movie or TV shows that share actors with a given show. In this way, the tool can recommend movie or TV shows to the user once the user provides their favorite, based on information on IMDb. <br>
<br>
### §1 Scrapy
To start a scrapy project, we need to run the following command in command prompt/terminal:


In [ ]:
conda activate PIC16B
scrapy startproject IMDB_scraper
cd IMDB_scraper

This creates the directory and files that we will need.

Now, we can start on wiritng the scraper script. Create `imdb_spider.py` in the `spiders` directory and set it up by entering the following code. I am going to use my favorite TV show, *Friends*, as an example, and the `start_urls` is its IMDb page.

In [ ]:
from urllib.request import Request
import scrapy

class ImdbSpider(scrapy.Spider):
    name = 'imdb_spider'
    
    start_urls = ['https://www.imdb.com/title/tt0108778/']

This creates a spider named `imdb_spide`. Then, we need to write parse functions for the `ImdbSpider` class so that the scraper can go through all actors in this TV show and obtain the names of all TV shows and movies that they have worked in. <br>
<br> Let's start with `parse(self,response)`. This function has one simple task, that is to take us to the "Full Cast & Crew" page, which has the url `<start_urls>fullcredits`. Once arrived at the full credits page, a second function, `parse_full_credits(self,response)`, is called in the `callback` argument of `scrapy.Request`. This is shown below:

In [ ]:
# Function to go to the full credits page
def parse(self,response):
        # Join current url with "fullcredits"
        full_credits = response.urljoin("fullcredits")
        yield scrapy.Request(full_credits, callback = self.parse_full_credits)

For the second function, `parse_full_credits(self,response)` needs to go the actor page of all actors who worked in *Friends*. To find the url to the actor pages, we can make use of the web developer tool:

As seen in the screen shot, the url to the actor page is the `"href"` attribute in a in the element `"td.primary_photo a"`. And with experimenting in scrapy shell (run `scrapy shell <start_urls>fullcredits` in terminal), it is clear that this information can be extracted using `response.css`. Once got to the actor page, the third function `parse_actor_page(self, response)` is called, again in the `callback` argument of `scrapy.Request`.

In [ ]:
# Function to go to all actor pages
def parse_full_credits(self,response):
    # List comprehension to obtain all urls to actor pages
    actor_pages = [a.attrib["href"] for a in response.css("td.primary_photo a")]
    for page in actor_pages:
        actcor_page = response.urljoin(page)
        yield scrapy.Request(actcor_page, callback = self.parse_actor_page)

Last, we need to write the function `parse_actor_page(self, response)`. This function yield dictionaries with the actor's name and all the movies and TV shows that the actor has been in. To find how we can obtain the two pieces of information, we need to use the web developer tool again:

Both of them are text information and can be obtained using `.get()` method. Again with experimenting in scrapy shell, it is found that the `actor_name` is in the class `"h1.header span.itemprop"` and the `movie_or_TV_name` is in `"div.filmo-row b a"` element. `set()` is used to avoid multiple entries, which could happen when the actors also acts as producer or other roles. This function is shwon below:

In [ ]:
# Function to yield actor_name and movie_or_TV_name in dictionaries 
def parse_actor_page(self, response):
        actor_name = response.css("h1.header").css("span.itemprop::text").get()
        # use set to avoid multiple entries
        movie_or_TV_list = set([a.get() for a in response.css("div.filmo-row b").css("a::text")])
        for a in movie_or_TV_list:
            movie_or_TV_name = a
            yield {"actor" : actor_name, "movie_or_TV_name" : movie_or_TV_name}

Now, the spider is ready! We can run it and save the results to `results.csv` by using the following command:

In [ ]:
scrapy crawl imdb_spider -o results.csv

### §2 Making Recommendations
Since we already obtained the table of all movies and TV shows that all actors/actresses in *Friends* have worked in, we can now find the ones that share the most actors/actresses with *Friends*.

In [1]:
import pandas as pd

We can read `results.csv` as a pandas data frame, and convert the movie_or_TV_name column to a set. Then, we can just simply count the number of times the movie or TV name has appeared in the results data frame by using `.sum()` method, and record the numbers by list comprehension:

In [3]:
# import results.csv
results = pd.read_csv("results.csv", sep = ',')
# convert to set to avoid multiple entries and back to list for iterable
movies_list = list(set(results["movie_or_TV_name"].tolist()))
# record the number of shared actors by list comprehension
num_of_shared_actors = [(results.movie_or_TV_name == movie).sum() for movie in movies_list]

Finally, we can make a new data frame with the lists `movies_list` and `num_of_shared_actors`, and sort the table by descending numbers of shared actors.

In [5]:
# Create and sort data frame for recommendations
recommendation = pd.DataFrame(data={"movie_or_TV_name":movies_list, "number_of_shared_actors":num_of_shared_actors})
recommendation = recommendation.sort_values(by="number_of_shared_actors", ascending=False )
# Tidying up the Table
recommendation = recommendation.reset_index()
recommendation = recommendation.drop("index", axis =1)
# Show the top 20 reccomedations
recommendation.head(20)

,movie_or_TV_name,number_of_shared_actors
0,Friends,854
1,ER,188
2,Entertainment Tonight,145
3,CSI: Crime Scene Investigation,135
4,Today,125
5,Seinfeld,124
6,The Tonight Show with Jay Leno,116
7,NCIS,113
8,Jimmy Kimmel Live!,110
9,NYPD Blue,107


Of course, the number 1 is *Friends* itself! <br>
<br>
The codes and .csv files can be found in my GitHub repository: https://github.com/ahuang0118/blog-post-3